In [1]:
import pandas as pd
import sqlite3
import json

#cargaré los datos de los archivos JSON

import pandas as pd
import sqlite3
import json

# Cargar los datos desde archivos JSON
customers = pd.read_json("sample_analytics.customers.json")
accounts = pd.read_json("sample_analytics.accounts.json")

# Cargar y expandir el JSON de transacciones (está anidado)
with open("sample_analytics.transactions.json") as f:
    raw_transactions = json.load(f)

# Extraer y normalizar las transacciones por cuenta
transactions_expanded = []
for entry in raw_transactions:
    account_id = entry['account_id']  # id de cuenta que agrupa cada set de transacciones
    for tx in entry['transactions']:
        tx_flat = tx.copy()
        tx_flat['account_id'] = account_id
        if isinstance(tx['date'], dict):  # revisar si la fecha está en formato {'$date': ...}
            tx_flat['date'] = tx['date']['$date']
        transactions_expanded.append(tx_flat)

# Crear DataFrame de transacciones y limpiar
transactions = pd.DataFrame(transactions_expanded)
transactions['date'] = pd.to_datetime(transactions['date'], errors='coerce')  # convertir a datetime
transactions = transactions[['account_id', 'date', 'amount', 'transaction_code', 'symbol', 'price', 'total']]

#normalizaremos los clientes
customers['birthdate'] = customers['birthdate'].apply(lambda x: x['$date'] if isinstance(x, dict) else x)
customers['birthdate'] = pd.to_datetime(customers['birthdate'], errors='coerce')
customers = customers.explode("accounts").rename(columns={'accounts': 'account_id'})  # una fila por cuenta
customers_final = customers[['username', 'name', 'birthdate', 'account_id']]

# Normalizar cuentas
accounts['products'] = accounts['products'].apply(lambda x: ', '.join(x) if isinstance(x, list) else str(x))
accounts_final = accounts[['account_id', 'limit', 'products']]

# Crear base de datos SQLite y guardar las tablas
conn = sqlite3.connect("sample_analytics.db")
customers_final.to_sql("customers", conn, if_exists="replace", index=False)
accounts_final.to_sql("accounts", conn, if_exists="replace", index=False)
transactions.to_sql("transactions", conn, if_exists="replace", index=False)

88119

## Pregunta 1
¿Cuál es el promedio, mínimo, máximo y desviación estándar del límite de las cuentas de usuarios?

In [2]:
accounts_final["limit"].agg(['mean', 'min', 'max', 'std']).round(2)

,limit
mean,9955.90
min,3000.00
max,10000.00
std,354.75


## Pregunta 2
¿Cuántos clientes poseen más de una cuenta?

In [3]:
# Esta consulta se ejecuta sobre la base de datos SQLite usando pandas.read_sql_query().
# Permite obtener un DataFrame directamente desde una instrucción SQL.
query = """SELECT COUNT(DISTINCT username) AS clientes_con_mas_de_una_cuenta
FROM (
    SELECT username, COUNT(DISTINCT account_id) AS cuentas
    FROM customers
    GROUP BY username
    HAVING cuentas > 1
);"""
pd.read_sql_query(query, conn)

,clientes_con_mas_de_una_cuenta
0,414
